In [3]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
symbol_groups = [list(stocks.Ticker)[x:x+100] for x in range(0, len(list(stocks.Ticker)), 100)]
symbol_strings = [(',').join(i) for i in symbol_groups]

In [50]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [41]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)
rv_dataframe['Ticker'] = stocks['Ticker']

In [16]:
data = {}
for s in symbol_strings:
    batch_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={s}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data.update(requests.get(batch_url).json())

In [42]:
rv_dict = {
    'Price': ['quote', 'latestPrice'],
    'Price-to-Earnings Ratio': ['quote', 'peRatio'],
    'Price-to-Book Ratio': ['advanced-stats', 'priceToBook'],
    'Price-to-Sales Ratio': ['advanced-stats', 'priceToSales'],
    'enterprise_value': ['advanced-stats', 'enterpriseValue'],
    'ebitda': ['advanced-stats', 'EBITDA'],
    'gross_profit': ['advanced-stats', 'grossProfit']
}

for k in rv_dict.keys():
    tmp = []
    for s in stocks['Ticker']:
        tmp.append(data[s][rv_dict[k][0]][rv_dict[k][1]])
    rv_dataframe[k] = tmp

In [43]:

for row in rv_dataframe.index:
        try:
            rv_dataframe.loc[row, 'EV/EBITDA']  = rv_dataframe['enterprise_value'][row]/rv_dataframe['ebitda'][row]
        except TypeError:
            rv_dataframe.loc[row, 'EV/EBITDA']  = np.NaN

        try:
            rv_dataframe.loc[row, 'EV/GP']  = rv_dataframe['enterprise_value'][row]/rv_dataframe['gross_profit'][row]
        except TypeError:
            rv_dataframe.loc[row, 'EV/GP']  = np.NaN
        
            

In [44]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [45]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

In [46]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

In [47]:
rv_dataframe.sort_values(by = 'RV Score', ascending=False, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

In [51]:
portfolio_input()

In [52]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])

In [53]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,enterprise_value,ebitda,gross_profit
0,NOW,565.71,353,512.20,1.0,31.53,0.962227,19.68,0.992048,157.237320,0.998012,25.318781,0.980119,0.986481,1.153979e+11,7.339092e+08,4.557799e+09
1,NVDA,230.57,867,73.11,0.950298,24.51,0.956262,24.70,1.0,59.175584,0.984095,35.446151,0.994036,0.976938,5.811419e+11,9.820636e+09,1.639506e+10
2,DXCM,420.25,475,267.77,0.994036,17.96,0.932406,16.52,0.970179,103.899207,0.996024,23.918476,0.968191,0.972167,4.096233e+10,3.942507e+08,1.712581e+09
3,PAYC,314.87,635,96.91,0.970179,21.50,0.94831,18.20,0.982107,57.463269,0.982107,20.500947,0.950298,0.9666,1.925701e+10,3.351186e+08,9.393230e+08
4,IDXX,550.71,363,63.49,0.936382,67.80,0.99006,14.41,0.954274,43.916205,0.962227,23.939456,0.970179,0.962624,4.637060e+10,1.055888e+09,1.936995e+09
5,FTNT,295.31,677,80.41,0.960239,59.58,0.986083,14.43,0.956262,60.945569,0.986083,17.430369,0.916501,0.961034,4.553388e+10,7.471237e+08,2.612330e+09
6,TYL,428.11,467,112.81,0.976143,7.72,0.795229,11.06,0.920477,79.812602,0.992048,25.076658,0.978131,0.932406,1.864021e+10,2.335497e+08,7.433292e+08
7,ISRG,279.10,716,60.37,0.924453,8.70,0.811133,18.08,0.978131,45.306195,0.968191,23.975105,0.972167,0.930815,9.958562e+10,2.198057e+09,4.153709e+09
8,CDNS,154.20,1297,61.40,0.928429,15.89,0.918489,14.00,0.950298,50.438424,0.974155,15.201100,0.87674,0.929622,4.204399e+10,8.335707e+08,2.765852e+09
9,MKTX,363.05,550,52.81,0.898608,13.01,0.87674,19.69,0.994036,34.139093,0.934394,18.732334,0.93837,0.928429,1.349469e+10,3.952855e+08,7.203954e+08
